# Parameter files

Struphy parameter files are Python scripts (.py) that can be executed with the Python interpreter.
For each `MODEL`, the default parameter file can be generated from the console via

```
struphy params MODEL
```

This will create a file `params_MODEL.py` in the current working directory. To run the model type

```
python params_MODEL.py
```

The user should modify the parameter file to launch a specific simulation.
As an example, in what follows we discuss the parameter file of the model [Vlasov](https://struphy.pages.mpcdf.de/struphy/sections/subsections/models_toy.html#struphy.models.toy.Vlasov) and run some simple examples. The file can be generated from

```
struphy params Vlasov
```

To see its contents, open the file in your preferred editor or type

```
cat params_Vlasov.py
```

## Part 1: imports

In [ ]:
from struphy.io.options import EnvironmentOptions, Units, Time
from struphy.geometry import domains
from struphy.fields_background import equils
from struphy.topology import grids
from struphy.io.options import DerhamOptions
from struphy.io.options import FieldsBackground
from struphy.initial import perturbations
from struphy.kinetic_background import maxwellians
from struphy.pic.utilities import LoadingParameters, WeightsParameters, BoundaryParameters
from struphy import main

# import model, set verbosity
from struphy.models.toy import Vlasov as Model
verbose = True

In [ ]:
# environment options
env = EnvironmentOptions()

# units
units = Units()

# time stepping
time_opts = Time(dt=0.2, Tend=10.0)

# geometry
l1 = -5.0
r1 = 5.0
l2 = -7.0
r2 = 7.0
l3 = -1.0
r3 = 1.0
domain = domains.Cuboid(l1=l1, r1=r1, l2=l2, r2=r2, l3=l3, r3=r3)

# fluid equilibrium (can be used as part of initial conditions)
equil = None

# grid
grid = None

# derham options
derham_opts = None

In [ ]:
# light-weight model instance
model = Model()

# species parameters
model.kinetic_ions.set_phys_params()

loading_params = LoadingParameters(Np=15)
weights_params = WeightsParameters()
boundary_params = BoundaryParameters(bc=('reflect', 'reflect', 'periodic'))
model.kinetic_ions.set_markers(loading_params=loading_params, 
                               weights_params=weights_params,
                               boundary_params=boundary_params)
model.kinetic_ions.set_sorting_boxes()
model.kinetic_ions.set_save_data(n_markers=1.0)

In [ ]:
# propagator options
model.propagators.push_vxb.set_options()
model.propagators.push_eta.set_options()

In [ ]:
# initial conditions (background + perturbation)
perturbation = None

background = maxwellians.Maxwellian3D(n=(1.0, perturbation))
model.kinetic_ions.var.add_background(background)

In [ ]:
main.run(model, 
         params_path=None, 
         env=env, 
         units=units, 
         time_opts=time_opts, 
         domain=domain, 
         equil=equil, 
         grid=grid, 
         derham_opts=derham_opts, 
         verbose=verbose, 
         )

## Post processing

In [ ]:
import os

path = os.path.join(os.getcwd(), "sim_1")
main.pproc(path, physical=True)

## Viewing particle orbits

In [ ]:
simdata = main.load_data(path)

In [ ]:
for k, v in simdata.pic_species["kinetic_ions"]["orbits"].items():
    print(f"{k = }, {type(v) = }")

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure()
ax = fig.gca()

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red']

time = 0.
dt = time_opts.dt
Tend = time_opts.Tend
for k, v in simdata.pic_species["kinetic_ions"]["orbits"].items():
    # print(k, v)
    alpha = (Tend - time)/Tend
    for i, particle in enumerate(v):
        ax.scatter(particle[1], particle[2], c=colors[i % 4], alpha=alpha)
    time += dt
    
ax.plot([l1, l1], [l2, r2], 'k')
ax.plot([r1, r1], [l2, r2], 'k')
ax.plot([l1, r1], [l2, l2], 'k')
ax.plot([l1, r1], [r2, r2], 'k')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_xlim(-6.5, 6.5)
ax.set_ylim(-9, 9)
ax.set_title(f'{int(Tend/dt)} time steps (full color at t=0)');